# Results Analysis

This notebook serves analysing LADDS run results, the following packages need to be installed

`conda install tqdm pykep pandas scikit-learn h5py -c conda-forge`

In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")
import math
from glob import glob

from tqdm import tqdm
import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import h5py
import yaml

from mpl_toolkits import mplot3d
#%matplotlib notebook

dt = 10 #timestep of the inspected simulations, affects time labels in plots
starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000') # starting t of the simulation
filter_CC = True # filter Constellation Collisions
nConstellations = 9
cnames = ["Debris","Astra1","Astra2","Astra3","OneWeb1","OneWeb2","Starlink1","Kuiper","Telesat","GuoWang"]

### Load the run data
Adapt the path below to the h5 file you want to load.

In [ ]:
#appending h5 files
#with h5py.File('../../14ySim/simulationData1C.h5', 'a') as hf:
#    file2 = h5py.File("../../14ySim/simulationData2A.h5",'r')
#    for pData in file2["ParticleData"].values():
#        hf.copy(pData,hf["ParticleData"])
#        
#    for cData in file2["CollisionData"].values():
#        hf.copy(cData,hf["CollisionData"])
    
#print("Finished")

In [ ]:
# Load hdf5 file
data = h5py.File("../../14ySim/simulationData1C.h5")

# Determine the iterations at which output was written
iterations_idx = list(data["ParticleData"].keys())
iterations_idx = [int(it) for it in iterations_idx]
iterations_idx.sort()
max_iterations = max(iterations_idx)
print("Found a total of", max_iterations, " iterations.")

# Find out the simulation runtime in days
end_t = pk.epoch(starting_t.mjd2000 + max_iterations * dt * pk.SEC2DAY)
total_days = end_t.mjd - starting_t.mjd
print("Simulation ran for a total of", total_days, " days.")

### Extract the positions, velocities and corresponding particle IDs from the hdf5

In [ ]:
rs,vs,ids = [],[],[] #will hold r,v, ids for whole simulation

for idx in tqdm(iterations_idx):
    
    # Load velocities
    v_x = data["ParticleData"][str(idx)]["Particles"]["Velocities"][:]["x"]
    v_y = data["ParticleData"][str(idx)]["Particles"]["Velocities"][:]["y"]
    v_z = data["ParticleData"][str(idx)]["Particles"]["Velocities"][:]["z"]
    v = np.vstack([v_x,v_y,v_z]).transpose()
    
    # Load positions
    r_x = data["ParticleData"][str(idx)]["Particles"]["Positions"][:]["x"]
    r_y = data["ParticleData"][str(idx)]["Particles"]["Positions"][:]["y"]
    r_z = data["ParticleData"][str(idx)]["Particles"]["Positions"][:]["z"]
    r = np.vstack([r_x,r_y,r_z]).transpose()
    
    # Get IDs
    ID = np.array(data["ParticleData"][str(idx)]["Particles"]["IDs"])
    
    # Store in single large list
    rs.append(r * 1000.) #convert to m 
    vs.append(v * 1000.) #convert to m 
    ids.append(ID)

In [ ]:
idCounts = [0] * (nConstellations+1)
ids_int = [int(id) for id in ids[len(iterations_idx)-1]]
for id in ids_int:
    idCounts[id // 1000000] = idCounts[id // 1000000] + 1
for i in range(len(idCounts)):
    print(str(i) + ": " + str(idCounts[i]) + "\n")

In [ ]:
#Finding constellation satellites
itera = 54
idxx = []
idss = []
added = 0
for i in range(len(ids[itera])):
    if(ids[itera][i] >= 6000000 and ids[itera][i] <= 7000000):
        idxx.append(i)
        idss.append(ids[itera][i])
        added += 1
        if(ids[itera][i] == 6002944):
            print("existiert")
        
print("done")

In [ ]:
#printing altitude of satellite with certain id
pos = [0] * 3
for i in range(len(idss)):
    idd = idss[i]
    if(idd == 6002944):
        pos[0] = rs[itera][idxx[i]][0] / 1000
        pos[1] = rs[itera][idxx[i]][1] / 1000
        pos[2] = rs[itera][idxx[i]][2] / 1000
        print(idxx[i])
        print(ids[itera][idxx[i]])
        print(np.linalg.norm(pos)-6371)

In [ ]:
#print ids and corresponding altitudes at iteration 0
added = 0
pos = [0] * 3
for i in idxx:
    pos[0] = rs[0][i][0] / 1000
    pos[1] = rs[0][i][1] / 1000
    pos[2] = rs[0][i][2] / 1000
    print(i)
    print(ids[0][i])
    print(np.linalg.norm(pos)-6371)
    added += 1

### Now some helper functions for later

In [ ]:
# How many iterations happened between IO
iteration_stepsize = iterations_idx[1] - iterations_idx[0]


def find_closest_it(array,value):
    """ Returns the index of the closest iteration to the passed iteration value. 
    E.g if saved iteration were [0,1000,2000] and you pass 500 , it will return 0, for 501 returns 1 etc."""
    idx = np.searchsorted(array, value+ (iteration_stepsize // 2), side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx - 1
    else:
        return idx - 1
    
def get_particle_r_v(ID,it):
    """From the large rs, vs arrays returns the values particle with a certain ID closest to the passed iteration"""
    it = find_closest_it(iterations_idx,it)
    idx = np.argmax(ids[it]==ID)
    return rs[it][idx],vs[it][idx]

In [ ]:
# Threshold for conjunction tracking as used in simulation
# Determines what is in the plots
thresholds = [1,5,10,20,25,50,100,250]

### Extract the conjunctions from the HDF5 file

In [ ]:
# Convert conjunctions to a pandas dataframe for convenience
conj = pd.DataFrame(columns=["P1","P2","Iteration","SquaredDistance"])
cconjs = []
for i in range(nConstellations+1):
    cconj = pd.DataFrame(columns=["P1","P2","Iteration","SquaredDistance"])
    cconjs.append(cconj)
collision_keys = data["CollisionData"].keys()
for it in tqdm(collision_keys):
    iteration = int(it)
    collisions = data["CollisionData"][it]["Collisions"]
    for collision in collisions:
        if(filter_CC and collision[0] // 1000000 != 0 and collision[1] // 1000000 != 0):
            continue
        centry = {"P1":collision[0],
                  "P2": collision[1],
                  "Iteration": iteration,
                  "SquaredDistance": collision[2]}
        conj = conj.append(centry,ignore_index=True)
        # also create buckets for each constellation
        cconjs[collision[0] // 1000000] = cconjs[collision[0] // 1000000].append(centry,ignore_index=True)
        cconjs[collision[1] // 1000000] = cconjs[collision[1] // 1000000].append(centry,ignore_index=True)
conj

### Clean up
Throw away all but the closest encounter, also compute actual distance (not squared)

In [ ]:
# Sort ascending by conjunction distance, then drop all but the first to get closest encounter
conj = conj.sort_values('SquaredDistance', ascending=True)
unique_conj = conj.drop_duplicates(subset=["P1","P2"],keep="first")
unique_conj["Distance"] = np.sqrt(unique_conj.SquaredDistance) * 1000. # Compute distance in meters
del conj # deleting to not accidentally use it

unique_cconjs = []
for i in range(nConstellations+1):
    unique_cconj = pd.DataFrame(columns=["P1","P2","Iteration","SquaredDistance"])
    unique_cconjs.append(unique_cconj)
    
#also clean up buckets
for i in range(len(cconjs)):
    cconjs[i] = cconjs[i].sort_values('SquaredDistance', ascending=True)
    unique_cconjs[i] = cconjs[i].drop_duplicates(subset=["P1","P2"],keep="first")
    unique_cconjs[i]["Distance"] = np.sqrt(unique_cconjs[i].SquaredDistance) * 1000. # Compute distance in meters

all_conj = unique_conj
unique_conj
print(len(unique_conj))

## Let the plotting begin.
We start with counting the number of conjunctions for different thresholds over the simulation time and then threshold vs total # of conjunctions.

In [ ]:
idss = [int(id) for id in ids[0]]
rss = [r for r in rs[0]]

min_altitude = 200
max_altitude = 2000
num_buckets = 100

bucket_size = (max_altitude - min_altitude) / num_buckets
altitude_axis = np.linspace(min_altitude,max_altitude,num_buckets)
buckets = [0] * num_buckets
for i in range(len(idss)):
    if(idss[i]<1000000):
        r = [rss[i][0]/1000,rss[i][1]/1000,rss[i][2]/1000]
        altitude = np.sqrt(r[0]*r[0]+r[1]*r[1]+r[2]*r[2]) - 6371
        if(altitude > min_altitude and altitude < max_altitude):
            buckets[int((altitude-min_altitude) // bucket_size)] += 1 

fig, ax = plt.subplots(figsize=(9,5),dpi=100)

ax.bar(altitude_axis,buckets,bucket_size+0.5,label="altitude")

ax.set_xlabel("Altitude (km)")
ax.set_ylabel("# of objects")
ax.set_title("distribution of objects by altitude")
plt.show()

print(buckets)

In [ ]:
#choose the plot base: all conjunctions (all_conj), debris-debris conjunctions (unique_cconj[0])
# constellation n (unique_cconjs[n])
unique_conj = all_conj

# determine time granularity
steps = 1000
t = np.linspace(0,end_t.mjd - starting_t.mjd,steps)
it = np.linspace(0,max_iterations,steps)
summed_conjs = [[] for _ in thresholds]

# Compute number of conjunctions
for i in tqdm(it):
    for idx,threshold in enumerate(thresholds):
        count = len(unique_conj[(unique_conj.Iteration < i) & (unique_conj.Distance < threshold)])
        summed_conjs[idx].append(count)

In [ ]:
fig = plt.figure(figsize=(9,5),dpi=100)
fig.patch.set_facecolor('white')

# Iterate over thresholds and plot for each
for idx,row in enumerate(thresholds):
    plt.plot(t,summed_conjs[idx],linewidth=3)
    
plt.legend([str(t) + "m" for t in thresholds],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
plt.title("Conjunction Thresholds Comparison")
plt.xlabel("Days")
plt.ylabel("# of Conjunctions")
plt.gca().set_yscale("log")
plt.tight_layout()

## Stack Bar plot

In [ ]:
sb_threshold = 100

steps = 100
t = np.linspace(0,end_t.mjd - starting_t.mjd,steps)
it = np.linspace(0,max_iterations,steps)
summed_conjs = [[] for _ in range(nConstellations+1)]
stepsizeSB = (end_t.mjd - starting_t.mjd)/steps + 0.5
for i in tqdm(it):
    for c_idx in range(nConstellations+1):
        count = len(unique_cconjs[c_idx][(unique_cconjs[c_idx].Iteration < i)&
                                         (unique_cconjs[c_idx].Distance < sb_threshold)])
        summed_conjs[c_idx].append(count)

fig, ax = plt.subplots(figsize=(9,5),dpi=100)

acc = np.array(summed_conjs[1])
ax.bar(t,summed_conjs[1],stepsizeSB,label=cnames[1])
for c_idx in range(2,10):
    ax.bar(t,summed_conjs[c_idx],stepsizeSB,label=cnames[c_idx],bottom=acc)
    acc = acc + np.array(summed_conjs[c_idx])

ax.set_xlabel("Days")
ax.set_ylabel("# of Conjunctions (" + str(sb_threshold) + "m)")
ax.set_title("Total Conjunctions by Constellation")
ax.legend()
plt.show()
print(len(unique_cconjs[2][(unique_cconjs[2].Iteration < it[12])]))
print(summed_conjs[0])

In [ ]:
# Compute conjunction counts at specific points in the simulation
steps = 1000 # threshold granularity
max_threshold = 10 # maximum investigated threshold
# Time axis
threshold_grid = np.logspace(-1,np.log10(max_threshold),steps)
sums = []
for idx,threshold in enumerate(threshold_grid):
    count = len(unique_conj[(unique_conj.Distance < threshold)])
    sums.append(count)

fig = plt.figure(figsize=(5,5),dpi=100)
fig.patch.set_facecolor('white')

plt.plot(threshold_grid,sums,linewidth=3)
    
plt.title("Conjunction Thresholds vs. \n # of Conjunctions")
plt.xlabel("Threshold [m]")
plt.ylabel("# of Conjunctions")
# plt.gca().set_xscale("log")
# plt.gca().set_yscale("log")
plt.tight_layout()

### Let's investigate the relationship of orbital elements and conjunctions (below some threshold)

In [ ]:
elements = [[],[],[],[],[],[]]
threshold = 200 #cutoff for investigated conjunctions
abc = 0
for idx,row in tqdm(unique_conj.iterrows(),total=len(unique_conj.index)):
    if row.Distance < threshold:
        abc += 1
        r,v = get_particle_r_v(row.P1,row.Iteration)
        if abc == 3:
            print(row.P1)
            print(r)
            print(v)
            print(row.P2)
            print(r)
            print(v)
        a,e,i,W,w,E = pk.ic2par(r.astype("double"),v.astype("double"), pk.MU_EARTH)
        elements[0].append(abs(a))
        elements[1].append(abs(e))
        elements[2].append(abs(i))
        elements[3].append(abs(W))
        elements[4].append(abs(w))
        elements[5].append(abs(E))
        
        r,v = get_particle_r_v(row.P2,row.Iteration)
        a,e,i,W,w,E = pk.ic2par(r.astype("double"),v.astype("double"), pk.MU_EARTH)
        elements[0].append(abs(a))
        elements[1].append(abs(e))
        elements[2].append(abs(i))
        elements[3].append(abs(W))
        elements[4].append(abs(w))
        elements[5].append(abs(E))
    
print("Collected data for ",len(elements[0])," conjunctions.")

In [ ]:
# compute orbital elements for the entire population at t=0 as a reference
pop_elements = [[],[],[],[],[],[]]
for r_i,v_i in tqdm(zip(rs[0],vs[0])):
    a,e,i,W,w,E = pk.ic2par(r_i.astype("double"),v_i.astype("double"), pk.MU_EARTH)
    pop_elements[0].append(abs(a))
    pop_elements[1].append(abs(e))
    pop_elements[2].append(abs(i))
    pop_elements[3].append(abs(W))
    pop_elements[4].append(abs(w))
    pop_elements[5].append(abs(E))

In [ ]:
# Plot histogram of collisions for each orbital element
for idx,element in enumerate(["a [m]","e","i [rad]","W","w","E"]):
    fig = plt.figure(figsize=(6,4),dpi=100)
    fig.patch.set_facecolor('white')
    bins = np.linspace(min(pop_elements[idx]),max(pop_elements[idx]),32)
    plt.hist(elements[idx],bins=bins,density = True,alpha=0.5)
    plt.hist(pop_elements[idx],bins=bins,density = True,alpha=0.5)
    plt.legend([f"Particles in conjunctions (< {threshold}m)","All particles at t=0"])
    plt.title("Histogram of "+element)
    plt.xlabel(element)
    plt.ylabel("Relative Frequency")
    plt.tight_layout()

In [ ]:
elements

### Now let's orbital elements of all particles over the simulation as a sanity check

In [ ]:
# Compute min, max and mean of orbital elements over simulation
min_elements = [[],[],[],[],[],[]]
mean_elements = [[],[],[],[],[],[]]
max_elements = [[],[],[],[],[],[]]

steps = list(range(len(rs)))

for idx in tqdm(steps[::1]): #pick every n-th step
    elements = [[],[],[],[],[],[]]
    v_it,r_it = vs[idx],rs[idx]
    for v,r in zip(v_it,r_it):
        a,e,i,W,w,E = pk.ic2par(r.astype("double"),v.astype("double"), pk.MU_EARTH)
        elements[0].append(abs(a))
        elements[1].append(abs(e))
        elements[2].append(abs(i))
        elements[3].append(abs(W))
        elements[4].append(abs(w))
        elements[5].append(abs(E))
    for i in range(6):
        min_elements[i].append(np.min(elements[i]))
        mean_elements[i].append(np.mean(elements[i]))
        max_elements[i].append(np.max(elements[i]))

In [ ]:
# Time axis of the simulation
t = np.linspace(0,end_t.mjd - starting_t.mjd,len(mean_elements[0]))

# Plot for each orbital element
for idx,element in enumerate(["a","e","i","W","w","E"]):
    fig = plt.figure(figsize=(6,4),dpi=100)
    fig.patch.set_facecolor('white')
    plt.plot(t,mean_elements[idx],linewidth=1)
#     plt.plot(t,min_elements[idx],linewidth=1)
#     plt.plot(t,max_elements[idx],linewidth=1)
#     plt.legend(["mean","min","max"],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
    plt.title("Evolution of "+element)
    plt.xlabel("Days")
    plt.ylabel("Mean " + element)
    plt.tight_layout()
#         plt.gca().set_yscale("log")

### Finally, we will plot the distribution of the distance to the next particle over time

In [ ]:
# Compute a KNN to get distance to nearest neighbors over simulation

all_distances = []

r_subset = rs[::1] #pick every n-th step

for r_it in tqdm(r_subset,total=len(r_subset)):
    elements = [[],[],[],[],[],[]]
    knn = NearestNeighbors(n_neighbors=2).fit(r_it)
    distances,_ = knn.kneighbors(r_it)
    distances = distances[:,1] / 1000 # convert to km
    all_distances.append(distances)
    
all_distances = np.asarray(all_distances)

In [ ]:
# Subsample to look only at those below some threshold
small_distances = []
max_dist = 25
for dist in all_distances:
    small_distances.append(dist[dist < max_dist])

In [ ]:
# Compute distributions for each iteration
bins = 64
x = np.linspace(0, max_dist, bins)
y = np.linspace(0, total_days, len(all_distances))

X, Y = np.meshgrid(x, y)
Z = []
for dist in tqdm(small_distances):
    hist,bin_vals = np.histogram(dist, bins = x,density=False)
    hist = np.cumsum(hist)
    hist = np.concatenate([hist,[hist[-1]]]) # last value remains for bucket
    Z.append(hist)
x = np.concatenate([[0],x])
Z = np.asarray(Z)

In [ ]:
# Create beautiful plots
fig = plt.figure(figsize = (8,8),dpi=100)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z,  rstride=1, cstride=1, cmap="plasma", edgecolor='none')
ax.view_init(elev=20., azim=120)
ax.set_xlabel('Closest Distance [km]')
ax.set_ylabel('Days')
ax.set_zlabel('Cumulative Frequency');
ax.set_xlim([0,max_dist])
plt.tight_layout()